In [6]:
import numpy as np
from scipy.sparse import identity, diags, csc_matrix  # type: ignore
from scipy import sparse  # type: ignore
from scipy.sparse.linalg import expm_multiply  # type: ignore
from typing import Tuple, Literal, List, Optional

prepare the matrices

In [46]:
spin_length = 1 / 2
qudit_range = np.arange(spin_length, -(spin_length + 1), -1)
lx = csc_matrix(
        1
        / 2
        * diags(
            [
                np.sqrt(
                    [
                        (spin_length - m + 1) * (spin_length + m)
                        for m in qudit_range[:-1]
                    ]
                ),
                np.sqrt(
                    [(spin_length + m + 1) * (spin_length - m) for m in qudit_range[1:]]
                ),
            ],
            [-1, 1],
        )
    )
ly = csc_matrix(
        1
        / (2 * 1j)
        * diags(
            [
                np.sqrt(
                    [
                        (spin_length - m + 1) * (spin_length + m)
                        for m in qudit_range[:-1]
                    ]
                ),
                -1
                * np.sqrt(
                    [(spin_length + m + 1) * (spin_length - m) for m in qudit_range[1:]]
                ),
            ],
            [-1, 1],
        )
    )

lz = csc_matrix(diags([qudit_range], [0]))
nocc = csc_matrix(diags([-qudit_range + 1 / 2], [0]))

and get the list done

In [47]:
def op_at_wire(op: csc_matrix, pos: int, dim_per_wire: List[int]) -> csc_matrix:
    """
    Applies an operation onto the wire and provides unitaries on the other wires.
    Basically this creates the nice tensor products.

    Args:
        op (matrix): The operation that should be applied.
        pos (int): The wire onto which the operation should be applied.
        dim_per_wire (int): What is the local Hilbert space of each wire.

    Returns:
        The tensor product matrix.
    """
    # There are two cases the first wire can be the identity or not
    if pos == 0:
        res = op
    else:
        res = csc_matrix(identity(dim_per_wire[0]))
    # then loop for the rest
    for i1 in np.arange(1, len(dim_per_wire)):
        temp = csc_matrix(identity(dim_per_wire[i1]))
        if i1 == pos:
            temp = op
        res = sparse.kron(res, temp)

    return res

In [48]:
print(lx)
print(ly)
print(lz)
print(nocc)

  (1, 0)	0.5
  (0, 1)	0.5
  (1, 0)	-0.5j
  (0, 1)	0.5j
  (0, 0)	0.5
  (1, 1)	-0.5
  (1, 1)	1.0


In [49]:
lx_list = []
ly_list = []
lz_list = []
nocc_list = []
n_wires = 2

spin_per_wire = 1 / 2 * np.ones(n_wires)
dim_per_wire = 2 * spin_per_wire + np.ones(n_wires)
print(dim_per_wire)
dim_per_wire = dim_per_wire.astype(int)
    
    
for i1 in np.arange(0, n_wires):
        # let's put together spin matrices
        lx_list.append(op_at_wire(lx, i1, list(dim_per_wire)))
        ly_list.append(op_at_wire(ly, i1, list(dim_per_wire)))
        lz_list.append(op_at_wire(lz, i1, list(dim_per_wire)))
        nocc_list.append(op_at_wire(nocc, i1, list(dim_per_wire)))

[2. 2.]


In [50]:
lx_list[1].todense()

matrix([[0. , 0.5, 0. , 0. ],
        [0.5, 0. , 0. , 0. ],
        [0. , 0. , 0. , 0.5],
        [0. , 0. , 0.5, 0. ]])

In [51]:

dim_hilbert = np.prod(dim_per_wire)
print(dim_hilbert)
int_matrix = csc_matrix((dim_hilbert, dim_hilbert))
for i1 in np.arange(0, n_wires):
        print(f"i1 = {i1}")
        for i2 in np.arange(i1 + 1, n_wires):
            print(f"i2 = {i2}")
            print(f"distance = {i1-i2}")
            int_matrix = (
                int_matrix + nocc_list[i1].dot(nocc_list[i2]) / np.abs(i1 - i2) ** 6
            )

4
i1 = 0
i2 = 1
distance = -1
i1 = 1


In [52]:
int_matrix.todense()

matrix([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 1.]])

In [71]:
initial_state = 1j * np.zeros(dim_per_wire[0])
initial_state[0] = 1 + 1j * 0
psi = sparse.csc_matrix(initial_state)
for i1 in np.arange(1, len(dim_per_wire)):
        initial_state = 1j * np.zeros(dim_per_wire[i1])
        initial_state[0] = 1 + 1j * 0
        psi = sparse.kron(psi, initial_state)
psi = psi.T


In [72]:
print(psi.todense())

[[1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]


time to apply the rotations to have both spins on the excited

In [73]:
psi = expm_multiply(-1j * np.pi * lx_list[0], psi)
print(nocc_list[0].dot(psi).todense())
print(nocc_list[1].dot(psi).todense())
psi = expm_multiply(-1j * np.pi * lx_list[1], psi)

print(psi.todense())
print(nocc_list[0].dot(psi).todense())
print(nocc_list[1].dot(psi).todense())

[[0.+0.j]
 [0.+0.j]
 [0.-1.j]
 [0.+0.j]]
[[0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]
[[ 7.95422958e-33+0.00000000e+00j]
 [ 0.00000000e+00-6.08209107e-17j]
 [ 0.00000000e+00-1.16332062e-16j]
 [-1.00000000e+00+0.00000000e+00j]]
[[ 0.+0.00000000e+00j]
 [ 0.+0.00000000e+00j]
 [ 0.-1.16332062e-16j]
 [-1.+0.00000000e+00j]]
[[ 0.+0.00000000e+00j]
 [ 0.-6.08209107e-17j]
 [ 0.+0.00000000e+00j]
 [-1.+0.00000000e+00j]]


print the state

In [74]:
print(nocc_list[0].dot(psi).todense())
print(nocc_list[1].dot(psi).todense())
print(int_matrix.dot(psi).todense())

[[ 0.+0.00000000e+00j]
 [ 0.+0.00000000e+00j]
 [ 0.-1.16332062e-16j]
 [-1.+0.00000000e+00j]]
[[ 0.+0.00000000e+00j]
 [ 0.-6.08209107e-17j]
 [ 0.+0.00000000e+00j]
 [-1.+0.00000000e+00j]]
[[ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [-1.+0.j]]
